# ガバナンスシートを登録する
研究データを検証するための指針となるガバナンスシートを登録するタスクです。

## ガバナンスシート登録フォームを表示する
ガバナンスシートを登録するためのフォームを表示します。<br>
登録済みのガバナンスシートが存在する場合は呼び出して表示します。<br>
ガバナンスシートに設定した内容が研究データの検証の指針となります。<br>
ガバナンスシートは検証を実施した単位で保存されますので、どのタイミングでどのガバナンスシートで検証を行ったかは振り返ることができます。<br>
研究の進捗や公開の範囲に合わせて設定内容を編集してください。<br>


In [ ]:
# ガバナンスシート登録フォームを表示する
import json
import os
import traceback

import panel as pn
from IPython.display import display, clear_output
from IPython.core.display import Javascript
from requests.exceptions import RequestException

from library.task_director import TaskDirector
from library.utils import dg_web
from library.utils.config import message as msg_config
from library.utils.error import (UnusableVault, ProjectNotExist,
                                 UnauthorizedError, PermissionError)
from library.utils.file import JsonFile
from library.utils.input import get_grdm_connection_parameters
from library.utils.storage_provider import grdm
from library.utils.widgets import Button


notebook_name = 'governance_sheet.ipynb'


class GovSheet(TaskDirector):

    def __init__(self, working_path:str) -> None:
        super().__init__(working_path, notebook_name)

        self.schema_form = dg_web.Form()

        # フォームボックス
        self._form_box = self.schema_form.form_box
        self._form_box.width = 900
        # メッセージ用ボックス
        self._msg_output = self.schema_form.msg_output
        self._msg_output.width = 900

    def get_grdm_params(self):
        """GRDMのトークンとプロジェクトIDを取得する"""
        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters()
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except PermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except ProjectNotExist as e:
            self._msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("1")
    def generateFormScetion(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        try:
            # パラメータの取得
            self.token, self.project_id = self.get_grdm_params()
            self.remote_path = dg_web.GOVSHEET_PATH
            self.api_url = grdm.API_V2_BASE_URL
            self.schema = None
            clear_output()

            # データの取得
            if self.token and not self._msg_output.has_message():
                data = self.get_data(
                    token=self.token, api_url=self.api_url,
                    project_id=self.project_id, remote_path=self.remote_path
                )
                self.schema = self.get_schema()

            # フォーム定義
            if self.schema and not self._msg_output.has_message():
                self.schema_form.create_widgets(self.schema, data)
                self.submit_button_title = msg_config.get('save', 'submit')
                self.submit_button = Button(width=500)
                self.submit_button.set_looks_init(self.submit_button_title)
                self.submit_button.on_click(self.submit)
                self._form_box.append(self.submit_button)
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def get_schema(self):
        """jsonschemaを取得する"""
        schema = {}
        try:
            schema = dg_web.get_govsheet_schema(
                scheme=dg_web.SCHEME, domain=dg_web.DOMAIN
            )
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return schema

    def get_data(self, token, api_url, project_id, remote_path):
        """指定したファイルの内容を取得する"""
        data = None
        try:
            data = grdm.download_json_file(
                token=token, api_url=api_url,
                project_id=project_id, remote_path=remote_path
            )
        except FileNotFoundError:
            data = None
        except json.JSONDecodeError:
            data = {}
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        return data

    @TaskDirector.callback_form('ガバナンスシートを保存する')
    def submit(self, event):
        file_path = os.path.join(self._abs_root_path, self.remote_path)
        tmpfile = JsonFile(file_path)
        try:
            data = self.schema_form.get_data()
            tmpfile.write(data)
            self.submit_button.disabled = True
            self.submit_button.set_looks_processing(msg_config.get('save', 'doing'))
            grdm.sync(
                    token=self.token,
                    api_url=self.api_url,
                    project_id=self.project_id,
                    abs_source = file_path,
                    abs_root=self._abs_root_path
                )
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(traceback.format_exc())
            return
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
            return
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
            return
        finally:
            tmpfile.remove(missing_ok=True)
            self.submit_button.disabled = False
            self.submit_button.set_looks_init(self.submit_button_title)

        self._form_box.clear()
        self._msg_output.update_success(msg_config.get('dg_web', 'saved_govsheet'))
        # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
        self.done_task()


GovSheet(os.path.abspath('__file__')).generateFormScetion()

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os
from library.task_director import TaskDirector

notebook_name = 'governance_sheet.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()